<a href="https://colab.research.google.com/github/DatawizzAI/Blogs/blob/main/Qwen_Grpo_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Qwen-0.5B GRPO Training on Taylor Swift QA Dataset

# **1. Setting Up the Environment**

# Install necessary dependencies

In [1]:
!pip install --no-cache-dir vllm
!pip install trl datasets
!pip install rouge-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 293.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 284.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 289.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 309.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 200.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 274.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 320.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 253.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.9/396.9 kB 289.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 301.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 285.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 257.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# **2. Importing Required Libraries**


In [2]:
import re
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import GRPOConfig, GRPOTrainer
from rouge_score import rouge_scorer
from peft import LoraConfig

# **3. Defining the Prompt and Dataset Preparation**


In [3]:
# Define a system prompt to maintain response consistency
SYSTEM_PROMPT = """You are a Taylor Swift expert. Answer CORRECTLY and CONCISELY questions about Taylor Swift's life, achievements, songs, and more."""

dataset_name = "lamini/taylor_swift"

def get_data(dataset_name, split="train") -> Dataset:
    """Loads and formats the dataset into a structured prompt format."""
    data = load_dataset(dataset_name)[split]
    data = data.map(lambda x: {
        'prompt': [
            {'role': 'system', 'content': SYSTEM_PROMPT},
            {'role': 'user', 'content': x['question']}
        ],
        'answer': x['answer']
    })
    return data.select_columns(['prompt', 'answer'])

dataset = get_data(dataset_name=dataset_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/573 [00:00<?, ?B/s]

(…)-00000-of-00001-54dd04266a81dbc7.parquet:   0%|          | 0.00/257k [00:00<?, ?B/s]

(…)-00000-of-00001-185d72ed4b72e469.parquet:   0%|          | 0.00/46.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/783 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/87 [00:00<?, ? examples/s]

Map:   0%|          | 0/783 [00:00<?, ? examples/s]

# **4. Defining Reward Functions**

In [4]:
# ROUGE-L based reward function to evaluate response similarity to reference answers
def rouge_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    responses = [completion[0]['content'] for completion in completions]
    rewards = [scorer.score(ref_answer, response)['rougeL'].fmeasure for response, ref_answer in zip(responses, answer)]
    return rewards

# Reward function based on length similarity
def length_similarity_reward_func(prompts, completions, answer, scale_factor=0.5, **kwargs) -> list[float]:
    responses = [completion[0]['content'] for completion in completions]
    rewards = [
        min(len(response.split()), len(ref_answer.split())) / max(len(response.split()), len(ref_answer.split())) * scale_factor
        if len(ref_answer.split()) > 0 else 0.0
        for response, ref_answer in zip(responses, answer)
    ]
    return rewards

def llm_judge_reward(prompt, generated_response, reference_answer, model, tokenizer): #llm_judge_score
    """Uses the fine-tuning Qwen-0.5B model to score responses locally."""
    eval_prompt = f"""
    Evaluate the correctness of the following response compared to the reference.

    Prompt: {prompt}
    Reference Answer: {reference_answer}
    Generated Response: {generated_response}

    Score the response between 0 and 1, where:
    - 1.0 is a perfect answer.
    - 0.0 is completely incorrect.

    Provide only a numerical score with no explanation.
    """

    # Tokenize input using the already-loaded tokenizer
    inputs = tokenizer(eval_prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure tensors are on the correct device

    # Generate response using the already-loaded model
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=10, do_sample=False)

    # Decode and extract the score
    score_text = tokenizer.decode(output[0], skip_special_tokens=True).strip()

    try:
        score = float(score_text)
        return max(0.0, min(1.0, score))  # Ensure the score is within the [0,1] range
    except:
        return 0.5  # Default fallback score if parsing fails

def llm_judge_reward_batch(prompts, generated_responses, answer, model, tokenizer):
    """Uses the fine-tuned Qwen-0.5B model to score multiple responses at once."""

    eval_prompts = [
        f"""Evaluate the correctness of the following response compared to the reference.

        Prompt: {p}
        Reference Answer: {r}
        Generated Response: {g}

        Score the response between 0 and 1. Only return a number."""
        for p, g, r in zip(prompts, generated_responses, answer)
    ]

    # Tokenize all prompts in a batch
    inputs = tokenizer(eval_prompts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # Generate responses in a batch
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10, do_sample=False)

    # Decode scores
    scores = [tokenizer.decode(o, skip_special_tokens=True).strip() for o in outputs]

    # Convert scores to float values
    try:
        return [max(0.0, min(1.0, float(s))) for s in scores]
    except:
        return [0.5] * len(prompts)  # Default fallback


def combined_reward(prompts, completions, answer, model, tokenizer):
    """Combines ROUGE, Length Similarity, and Qwen-0.5B as LLM-J."""

    rouge_scores = rouge_reward_func(prompts, completions, answer)
    length_scores = length_similarity_reward_func(prompts, completions, answer)

    generated_responses = [c[0]['content'] for c in completions]
    llm_scores = llm_judge_reward_batch(prompts, generated_responses, answer, model, tokenizer)

    # Weighted combination of scores
    final_rewards = [
        (0.3 * rouge) + (0.2 * length) + (0.5 * llm)
        #(0.4 * rouge) + (0.2 * length) + (0.4 * llm)
        #(0.2 * rouge) + (0.2 * length) + (0.6 * llm)
        for rouge, length, llm in zip(rouge_scores, length_scores, llm_scores)
    ]

    return final_rewards

# **5. Model and Tokenizer Initialization**


In [5]:
#!huggingface-cli login

In [6]:

model_name = "Qwen/Qwen2.5-0.5B-Instruct"
output_dir = "outputs/Qwen-0.5B-GRPO"

training_args = GRPOConfig(
    output_dir=output_dir,
    learning_rate=0.0001, #0.0002,
    adam_beta1=0.9,
    adam_beta2=0.99,
    weight_decay=0.1,
    warmup_steps=100, #10,
    lr_scheduler_type='cosine_with_restarts',  #'cosine',
    logging_steps=1,
    bf16=True,
    per_device_train_batch_size=8, #4,
    gradient_accumulation_steps=1,
    num_generations=8, #4,
    max_prompt_length=256,  #256,
    max_completion_length=128, #, 192, #256,
    num_train_epochs=3, #5,
    save_steps=100,
    log_on_each_node=False,
    use_vllm=True,
    vllm_gpu_memory_utilization=0.6,
    vllm_device="cuda:0",
    report_to="none"
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=None
).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# **6. Fine-Tuning with GRPO**
peft_config = LoraConfig(
    r=16, #32,
    lora_alpha=32, #64,
    target_modules="all-linear",
    task_type="CAUSAL_LM",
    lora_dropout=0.1, #=0.05,
)

trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    #reward_funcs=[rouge_reward_func, length_similarity_reward_func],
    reward_funcs=[lambda prompts, completions, answer: combined_reward(prompts, completions, answer, model, tokenizer)],  # Pass model & tokenizer
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config,
)

from transformers import TrainerCallback

class AdjustContextLengthCallback(TrainerCallback):
    """Dynamically increases max_completion_length during training."""

    def on_step_begin(self, args, state, control, **kwargs):
        """Adjusts max_completion_length based on training progress."""
        step = state.global_step

        if step >= 1000:
            args.max_prompt_length = 384  # Allow longer completions
        elif step >= 500:
            args.max_completion_length = 256 #320  # Gradually increase

        # Log changes
        if step in [500, 1000]:
            print(f"Adjusted max_completion_length to {args.max_completion_length} at step {step}")


# Add dynamic context adjustment
trainer.add_callback(AdjustContextLengthCallback())

trainer.train()

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

...

Adjusted max_completion_length to 256 at step 600


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarni

Adjusted max_completion_length to 256 at step 1000


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarni

TrainOutput(global_step=2349, training_loss=0.06481866186338059, metrics={'train_runtime': 3097.4967, 'train_samples_per_second': 0.758, 'train_steps_per_second': 0.758, 'total_flos': 0.0, 'train_loss': 0.06481866186338059})

# **7. Evaluating the Model on the Test Set**

In [7]:
import time

eval_dataset = get_data(dataset_name=dataset_name, split="test")
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
separator = ";Assistant:"

total_rouge_l = 0
total_inference_time = 0
for example in eval_dataset:
    start_time = time.time()  # Start timer
    prompt_text = "".join([d['content'] for d in example["prompt"]]) + " " + separator
    generated_text = trainer.model.generate(
        **trainer.processing_class(prompt_text, return_tensors='pt', padding=True).to('cuda')
    )
    generated_text = trainer.processing_class.batch_decode(generated_text, skip_special_tokens=True)[0]

    if separator in generated_text:
        generated_text = generated_text.split(separator, 1)[-1].strip()
    inference_time = time.time() - start_time  # Measure inference time
    total_inference_time += inference_time

    # Calculate ROUGE-L F1 score
    rouge_scores = scorer.score(example["answer"], generated_text)
    rouge_l_f1 = rouge_scores['rougeL'].fmeasure
    total_rouge_l += rouge_l_f1

average_rouge_l = total_rouge_l / len(eval_dataset)
average_inference_time = total_inference_time / len(eval_dataset)
print(f"Average ROUGE-L F1 score on test set: {average_rouge_l}")
print(f"Inference Time: {average_inference_time:.4f} sec\n")


Map:   0%|          | 0/87 [00:00<?, ? examples/s]

Average ROUGE-L F1 score on test set: 0.3173953729553082
Inference Time: 1.4007 sec



In [8]:
#Average ROUGE-L F1 score on test set: 0.3173953729553082 Inference Time: 1.4007 sec
#Average ROUGE-L F1 score on test set: 0.3108756580458049 Inference Time: 1.4187 sec
#Average ROUGE-L F1 score on test set: 0.2945748252841615
#Average ROUGE-L F1 score on test set: 0.3309647104626008  Inference Time: 1.4300 sec
#Average ROUGE-L F1 score on test set: 0.27408944466235735 (added context size callback)
#Average ROUGE-L F1 score on test set: 0.284495829735976 (no rouge score)
#Average ROUGE-L F1 score on test set: 0.3280152679299962
#Average ROUGE-L F1 score on test set: 0.24813469106304242 (added context size callback)
#Average ROUGE-L F1 score on test set: 0.31100436235529977
#Average ROUGE-L F1 score on test set: 0.3258220628998949
#Average ROUGE-L F1 score on test set: 0.3229716893012946
#Average ROUGE-L F1 score on test set: 0.3142907212008961
#Average ROUGE-L F1 score on test set: 0.31114052031334466
#Average ROUGE-L F1 score on test set: 0.31708317892042437